In [80]:
train = []
test = []
train_c = [
     ('I love this sandwich.', 'pos'),
     ('this is an amazing place!', 'pos'),
     ('I feel very good about these beers.', 'pos'),
     ('this is my best work.', 'pos'),
     ("what an awesome view", 'pos'),
     ('I do not like this restaurant', 'neg'),
     ('I am tired of this stuff.', 'neg'),
     ("I can't deal with this", 'neg'),
     ('he is my sworn enemy!', 'neg'),
     ('my boss is horrible.', 'neg'),
     ('She is my best friend.', 'pos'),
     ("I'm happy to have a new friend.", 'pos'),
     ("Stay thirsty, my friend.", 'pos'),
     ("He ain't from around here.", 'neg')
]

test_c = [
     ('the beer was good.', 'pos'),
     ('I do not enjoy my job', 'neg'),
     ("I ain't feeling dandy today.", 'neg'),
     ("I feel amazing!", 'pos'),
     ('Gary is a friend of mine.', 'pos'),
     ("I can't believe I'm doing this.", 'neg')
]

In [73]:
from preprocessor import process_line, get_words, stemmer, stop_words, get_lines
import os
import re
import random
stop_words = set({})
def merge_dicts(*dict_args):
    """
    Given any number of dicts, shallow copy and merge into a new dict,
    precedence goes to key value pairs in latter dicts.
    """
    result = {}
    for dictionary in dict_args:
        if bool(dictionary):
            for x in dictionary:
                if x not in stop_words:
                    if x not in result:
                        result[x] = 0
                    result[x] +=  dictionary[x];    
    return result

def naiive(freq_map, classes, doc, total_count, total_docs):
    words = get_words(doc)
    prob = {}
    max_class = ""
    max_prob = 0
    for c in classes:
        prob[c] = classes[c] / total_docs
        for x in words:
            x = stemmer.stemWord(x)
            if x not in stop_words and x in freq_map[c]:
                prob[c] = prob[c] * (freq_map[c][x]+1) / (total_count[c] + len( stop_words))
            elif x not in stop_words:
                prob[c] = prob[c] / (total_count[c] + len( stop_words))
        if max_prob < prob[c]:
            max_class = c
            max_prob = prob[c]
    return [max_class, max_prob]

In [81]:
classes = { 'talk.politics.misc' : 0, 'talk.religion.misc' : 0 }
dictionary = {}
words = []
keywords = set({})
for c in classes:
    files = os.listdir("./"+ c)
    no = 0
    for file in files:
        if no > 50:
            break;
        lines = []
        fp = open(c + "/" + file , "r")
        try:
            paras = fp.readlines()
            #print(paras)
            fp.close()
            for para in paras:
                para = re.sub("\n", "", para.strip())
                lines = lines + [line for line in get_lines(para) if line.strip()!=""]
            #process_paragraph(fp.readlines(), dictionary, lines, words, keywords)
            for line in lines:
                line =re.sub("[\d]+", " ", line)
                if line.strip() != "":
                    train.append((line,c))
            no += 1
        except:
            print("000")
test = random.sample(train, 40)            
print(len(train))

6584


In [82]:
#classes = {}
freq_map = {}
total_count = {}
total_docs = 0

for t in train:
    classes[t[1]] = 0
    freq_map[t[1]] = {}
    total_count[t[1]] = 0    

for t in train:
    total_docs += 1
    classes[t[1]] += 1
    dictionary = {}
    process_line(t[0], dictionary,[], set())
    freq_map[t[1]] = merge_dicts(freq_map[t[1]], dictionary)
    for x in dictionary:
        if x not in stop_words:
            total_count[t[1]] += dictionary[x]
    
print(classes)
print(total_count)
        

success =0
failure =0

for t in test:
    result = naiive(freq_map, classes, t[0], total_count, total_docs)
    print(t)
    print(result)
    if result[0] == t[1]:
        print("success")
        success += 1;
    else:
        print("failure")
        failure += 1;
print("**********************************************")
print("accuracy")
print(success / (success + failure))
#print(success / (success + failure)

{'talk.politics.misc': 3944, 'talk.religion.misc': 2640}
{'talk.politics.misc': 22941, 'talk.religion.misc': 15539}
("STEPHANOPOULOS:  We're working on it.", 'talk.politics.misc')
['talk.politics.misc', 1.8613704696392037e-16]
success
('good examples of "people"', 'talk.politics.misc')
['talk.politics.misc', 2.3613517385978068e-11]
success
('the situation', 'talk.politics.misc')
['talk.politics.misc', 2.304533773027442e-05]
success
('you ought to consider', 'talk.politics.misc')
['talk.politics.misc', 4.7026422062085444e-11]
success
(' Adda Wainwright        ', 'talk.politics.misc')
['talk.religion.misc', 1.6606112328521833e-09]
failure
('Q\t  Environmental groups have asked him to make a major', 'talk.politics.misc')
['talk.politics.misc', 6.846572246036901e-24]
success
('the Bosnia policy', 'talk.politics.misc')
['talk.politics.misc', 6.500017052259854e-10]
success
('stretching possible to include events such as the atoms in my terminal', 'talk.religion.misc')
['talk.religion.misc', 